In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
cc = CasosConfirmados()
# cc.read()
# cc.save()
cc.load()
notifica = Notifica()
# notifica.download_todas_notificacoes()
# notifica.save()
notifica.load()
print(cc.shape())
print(notifica.shape())

(581195, 10614, 425720, 141444)
(2296606, 1055528, 10936, 1227993, 2149)


In [3]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
print(casosn.shape)

(2296606, 55)


In [4]:
#pega o index dos casos confirmados que foram encontrados no notifica utilizando um dos hash
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
print(len(idx_casos_confirmados))
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
print(len(idx_casos_confirmados))
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()
print(len(idx_casos_confirmados))

#trasnforma essa lista de index em um conjunto, para deixar apenas os index unicos
idx_casos_confirmados = set(idx_casos_confirmados)
print(len(idx_casos_confirmados))

#realiza a diferença de conjunto para pegar os index dos que não foram encotrados no notifica
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape ## aqui o numero total é 27910 casos não encontrados no notifica

531996
1033553
1506798
553285


(27910, 23)

In [5]:
#separa os casos confirmados que tem correspondente nos casos notifica pelo hash que encontrou essa correspondencia
cchr = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])].copy()
ccha = casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])].copy()
cchd = casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])].copy()

In [6]:
#separa os casos do notifica que tem correspondente no casos confirmados pelo hash que encontrou essa correspondencia
cnhr = casosn.loc[casosn['hash_resid'].isin(casosc['hash_resid'])].copy()
cnha = casosn.loc[casosn['hash_atend'].isin(casosc['hash_atend'])].copy()
cnhd = casosn.loc[casosn['hash_diag'].isin(casosc['hash_diag'])].copy()

In [7]:
#realiza o merge com as partes separadas
mchr = pd.merge(cchr,cnhr[['id','hash_resid','manter']],on='hash_resid',how='left')
mcha = pd.merge(ccha,cnha[['id','hash_atend','manter']],on='hash_atend',how='left')
mchd = pd.merge(cchd,cnhd[['id','hash_diag','manter']],on='hash_diag',how='left')

print(mchr.shape)
print(mcha.shape)
print(mchd.shape)
print(mchr.shape[0] + mcha.shape[0] + mchd.shape[0])

(625174, 25)
(578885, 25)
(481549, 25)
1685608


In [25]:
#copia essas partes, concatenando e removendo os duplicados gerados no merge separado
mccn = mchr.copy()
mccn = mccn.append(mcha.loc[~mcha['ordem'].isin(mchr['ordem'])].copy())
mccn = mccn.append(mchd.loc[~mchd['ordem'].isin(mchr['ordem'].append(mcha['ordem']))].copy())

In [26]:
#remove os duplicados que havia mais de 1 correspondecia no notifica
mccn = mccn.drop(index=mccn.loc[(mccn.duplicated('ordem',keep=False)) & (mccn['manter']==1)].index)
mccn.shape

(571679, 25)

In [27]:
ordem_sem_notifica = casosc.loc[~casosc['ordem'].isin(mccn['ordem']),'ordem'].values.tolist()
print(len(ordem_sem_notifica))
casos_no_notifica_que_escaparam = set.difference(set(ordem_sem_notifica),set(casos_confirmados_nao_notifica['ordem'].values.tolist()))
casosc.loc[casos_no_notifica_que_escaparam]

31287


TypeError: 'DataFrame' object is not callable

In [11]:
mccn.shape[0] + casosc.loc[~casosc['ordem'].isin(mccn['ordem'])].shape[0] #no fim juntando tudo ficou casos a mais ???????????????????????????????

583982

In [12]:
writer = pd.ExcelWriter("mccn.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

mccn.to_excel(writer,sheet_name='mccn',index=None)
casosc.loc[~casosc['ordem'].isin(mccn['ordem'])].to_excel(writer,sheet_name='nmccn',index=None)
casosc.loc[~casosc['ordem'].isin(mccn['ordem'])].groupby('data_comunicacao')[['ordem']].count().to_excel(writer,sheet_name='totais')

writer.save()
writer.close()

KeyboardInterrupt: 